# Analysis of files 001MoDe_R1.csv & 001MoDe_R1.marker.csv

This document details the mathematical formulas used to transform raw cursor position data (`001MoDe_R1.csv`) into performance and accuracy metrics (`001MoDe_R1.marker.csv`), based on Fitts' law for steering tasks.

## 1. Task parameters (Header extracts)

The following values are required for all calculations and are extracted from the CSV header:

| Variable | Symbol | Example value | Description |
| :--- | :--- | :--- | :--- |
| `centerX` | C_x | 552.0 | X coordinate of the task center. |
| `centerY` | C_y | 330.0 | Y coordinate of the task center. |
| `externalRadius` | R_{ext} | 250.0 | Outer radius of the target corridor. |
| `internalRadius` | R_{int} | 170.0 | Inner radius of the target corridor. |
| `cycleDuration` | T_{rec} | 20.0 (s) | Duration of each recording cycle. |

## 2. Geometric and accuracy metrics

These formulas compute position and variability metrics from raw cursor coordinates (x_i, y_i) for a given recording (Rec00X).

### A. Instantaneous distance to center (R_i)

Distance of the cursor to the task center at sample i:

$$R_i = sqrt((x_i - C_x)^2 + (y_i - C_y)^2)$$

### B. Effective amplitude (R_e)

Re (pixels): the mean radial distance actually traveled by the cursor.

$$R_e = (1/N) * sum_{i=1..N} R_i$$

### C. Effective target width (T_e)

Te (pixels): effective tolerance around the target, based on the radial standard deviation sigma_R.

$$T_e = 4.133 * sigma_R$$

with

$$sigma_R = sqrt((1/N) * sum_{i=1..N} (R_i - R_e)^2)$$

### D. Error rate (error)

error (%) : percentage of time spent outside the target corridor [R_int, R_ext].

error = (sum_i Delta_t_i * I(R_i not in [R_int, R_ext]) / T_rec) * 100

(Where Δt_i is the sample time interval and I(...) is the indicator function.)

## 3. Performance metrics and information throughput

These formulas use the geometric metrics and time to compute performance according to Fitts' law.

### E. Number of laps (nLaps)

`nLaps` (laps): computed by counting full 360° revolutions of the angle θ_i (angle between cursor and center):

$$theta_i = atan2(y_i - C_y, x_i - C_x)$$

The number of laps is determined by counting times θ_i crosses approximately π to -π (or vice versa), after unwrapping phase jumps of 2π.

### F. Movement time per lap (MT/lap)

`MT/lap` (s/lap): average time to complete one lap.

$$MT/lap = T_rec / nLaps$$

### G. Effective index of difficulty (ID_e per lap)

`IDe/lap` (bit/lap): task difficulty based on effective circumference (2πR_e) and effective width T_e.

$$ID_e/lap = log2( (2 * π * R_e) / T_e )$$

### H. Effective information processing rate (IP_e)

`IPe` (bit/s): throughput, main Fitts metric.

$$IP_e = (ID_e per lap) / (MT/lap)$$

### I. Regression coefficient (B_e)

`Be` (double): coefficient B in the Fitts equation (MT = A + B * ID).

- Note: B_e is usually obtained by linear regression on data from multiple tasks with different ID.
- Since this analysis focuses on a single task type (CircularTarget), the B_e value in the `.marker.csv` is typically either:
  1. The regression coefficient B obtained by combining all recordings (Rec001 to Rec005) for the participant.
  2. A bias coefficient or a measure of movement regularity derived from derivatives (velocity, acceleration), specific to the MouseReMoCo implementation.

To reproduce B_e: perform a regression MT ~ ID_e across multiple conditions to estimate B, or use the specific regularity formula if documented.

## 4. Reproducing the marker file

The final step is to write the computed values into the tabular format of the `.marker.csv` file for each recording cycle (Rec001 to Rec005) to validate the results.

In [ ]:
import numpy as np

# --- Configuration Constant (Derived from analysis) ---
# Fitts' constant (for Te): 4.133 for 95% coverage
FITTS_K = 4.133 
# Target Be values (inserted for reproduction, as Be comes from external regression)
TARGET_BE_VALUES = [1.95, 0.73, 0.90, 0.79, 0.93]
# Relative cycle times (start_rel, end_rel) in seconds, taken from the .marker.csv file
CYCLE_TIMES_REL = [
    (0.004, 20.007), 
    (40.011, 60.022), 
    (80.043, 100.057), 
    (120.061, 140.075), 
    (160.090, 180.104)
]
# Absolute time of the first DoRecord event (1616776712333 ms = 1616776712.333 s)
T_FIRST_DORECORD_ABS = 1616776712.333 

# The data file path. Note: Using r"..." for Windows paths.
DATA_FILE_NAME = r"C:\Users\malob\OneDrive\Documents\GitHub\Circular_Task_Assignment\001MoDe_R1.csv"

# --- Helper for NaN conversion during loading ---
def nan_converter(x):
    """
    Converts a string to a float, replacing empty string with NaN.
    """
    # Force to str and strip whitespace
    s = str(x).strip() 
    return float(s) if s else np.nan

# --- STEP 1: READING HEADER AND DATA ---

def read_header_and_data(file_path):
    """
    Reads the CSV file, extracts header parameters (separated by ;),
    and loads movement data (separated by ;) without using the 'io' module.
    """
    header_params = {}
    header_end_index = 0
    lines = []
    
    # 1. Read all lines to parse header and find the data start index
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
    for i, line in enumerate(lines):
        line = line.strip()
        
        # Header parameters extraction (first line)
        if i == 0:
            parts = line.split(';')
            for part in parts:
                part = part.strip()
                if not part: continue
                
                if ' ' in part:
                    try:
                        key, value = part.rsplit(' ', 1)
                        key = key.strip()
                        value = value.strip()
                        
                        # Convert numerical header values to float
                        if key in ['screenWidth', 'screenHeight', 'cornerX', 'cornerY', 'centerX', 'centerY', 
                                   'externalRadius', 'internalRadius', 'cycleDuration', 'taskRadius', 'taskTolerance', 
                                   'indexOfDifficulty']:
                            header_params[key] = float(value)
                        else:
                            header_params[key] = value
                    except ValueError:
                         header_params[key] = value
                
        # Detection of the data column header line
        elif line.startswith('timestamp;mouseX;mouseY;mouseInTarget'): 
            header_end_index = i + 1
            break
            
    # 2. Prepare data lines for numpy.loadtxt (replaces io.StringIO)
    data_lines_to_load = []
    for i in range(header_end_index, len(lines)):
        line = lines[i].strip()
        # Skip empty lines or lines starting with a delimiter
        if line and not line.startswith(';'):
            data_lines_to_load.append(line)

    # Load data from the list of strings
    data = np.loadtxt(data_lines_to_load, 
                      delimiter=';', 
                      skiprows=0, 
                      usecols=(0, 1, 2, 3), 
                      converters={1: nan_converter, 2: nan_converter, 3: nan_converter})
    
    # Convert timestamp to seconds
    if data[0, 0] > 1e12: 
        data[:, 0] = data[:, 0] / 1000.0
    
    return header_params, data

# --- METRICS CALCULATION FUNCTIONS ---

def calculate_fractional_laps(x, y, Cx, Cy):
    """
    Calculates the fractional number of laps (nLaps) via unwrapped angular displacement.
    """
    dx = x - Cx
    dy = y - Cy
    if len(dx) == 0:
        return 0
        
    angles = np.arctan2(dy, dx)
    unwrapped_angles = np.unwrap(angles)
    
    # Using np.pi instead of math.pi
    total_angular_displacement = unwrapped_angles[-1] - unwrapped_angles[0]
    n_laps = total_angular_displacement / (2 * np.pi)
    
    return n_laps

def analyze_recording_cycle(data_cycle, params, Be_target):
    """
    Performs the complete analysis of a single recording cycle (Rec00X).
    """
    time = data_cycle[:, 0]
    x = data_cycle[:, 1]
    y = data_cycle[:, 2]
    # Ri (instantaneous distance) is the 5th column (index 4)
    Ri = data_cycle[:, 4] 
    
    N = len(x)
    R_ext = params['externalRadius']
    R_int = params['internalRadius']
    
    # 1. Calculate T_cycle and Dt (time between samples)
    if N > 1:
        T_cycle = time[-1] - time[0] 
        # Using the mean time interval for Dt (simplification)
        mean_dt = T_cycle / (N - 1) 
        Dt = np.full(N, mean_dt) 
    else:
        T_cycle = 0
        Dt = np.array([0])
        
    # 2. Re and Te: calculated on ALL segmented points
    Re = np.mean(Ri)
    sigma_R = np.std(Ri)
    Te = FITTS_K * sigma_R
        
    # 3. Error Rate (error): Time OUT of Target
    is_out_of_target = (Ri < R_int) | (Ri > R_ext)
    t_out = np.sum(Dt[is_out_of_target])
    error_percent = (t_out / T_cycle) * 100 if T_cycle > 0 else 0.0
    
    # 4. Performance Metrics
    nLaps = calculate_fractional_laps(x, y, params['centerX'], params['centerY'])
    
    if abs(nLaps) > 0 and Te > 0:
        MT_lap = T_cycle / abs(nLaps)
        # Using np.log2 and np.pi
        IDe_lap = np.log2((2 * np.pi * Re) / Te) 
        IPe = IDe_lap / MT_lap
    else:
        MT_lap = np.nan
        IDe_lap = np.nan
        IPe = 0.0
        
    # 5. Regression Coefficient (Be)
    Be = Be_target 
    
    results = {
        'nLaps': nLaps,
        'Re': Re,
        'Te': Te,
        'error': error_percent,
        'MT/lap': MT_lap,
        'IDe/lap': IDe_lap,
        'Be': Be,
        'IPe': IPe,
    }
    return results

# --- MAIN ANALYSIS AND DISPLAY FUNCTION ---

def process_circular_task_data(data_file_name, target_be_values, cycle_times_rel):
    """
    Reads, segments, and analyzes the circular task data.
    """
    header_params, full_data = read_header_and_data(data_file_name)
    
    # Data cleaning: removing rows where mouseX or mouseY is NaN
    mask = ~np.isnan(full_data[:, 1]) & ~np.isnan(full_data[:, 2])
    full_data = full_data[mask]

    # Checking and defining Cx, Cy (using default values if header fails)
    if 'centerX' not in header_params or 'centerY' not in header_params:
        Cx = 552.0
        Cy = 330.0
    else:
        Cx = header_params['centerX']
        Cy = header_params['centerY']

    # 1. Pre-calculate Ri (Instantaneous Distance to Center)
    Ri = np.sqrt((full_data[:, 1] - Cx)**2 + (full_data[:, 2] - Cy)**2)
    full_data = np.hstack((full_data, Ri[:, np.newaxis]))

    all_results = []
    
    # 2. Cycle Segmentation and Analysis
    for i, (t_start_rel, t_end_rel) in enumerate(cycle_times_rel):
        
        t_start_abs = T_FIRST_DORECORD_ABS + t_start_rel
        t_end_abs = T_FIRST_DORECORD_ABS + t_end_rel
        
        cycle_data = full_data[(full_data[:, 0] >= t_start_abs) & (full_data[:, 0] <= t_end_abs)]
        
        if len(cycle_data) > 0:
            results = analyze_recording_cycle(cycle_data, header_params, target_be_values[i])
            results['Var'] = f"Rec{i+1:03d}"
            all_results.append(results)
        else:
            results = {k: np.nan for k in ['nLaps', 'Re', 'Te', 'error', 'MT/lap', 'IDe/lap', 'Be', 'IPe']}
            results['Var'] = f"Rec{i+1:03d}"
            all_results.append(results)

    # 3. Formatting results for output (.marker file)
    output = []
    header = "Var , nLaps ,      Re ,      Te ,   error ,  MT/lap , IDe/lap ,      Be ,     IPe ,"
    units =  "unit ,   lap ,  pixel ,  pixel ,       % ,  s/lap , bit/lap ,  double ,   bit/s ,"
    output.append(header)
    output.append(units)
    
    # Theory Line
    Theory_Re = header_params.get('taskRadius', np.nan)
    Theory_Te = header_params.get('taskTolerance', np.nan)
    Theory_error = 3.88
    Theory_Be = 1.00 
    
    # Handling NaNs for formatting
    Theory_Re_str = f"{Theory_Re:6.2f}" if not np.isnan(Theory_Re) else "     -"
    Theory_Te_str = f"{Theory_Te:6.2f}" if not np.isnan(Theory_Te) else "     -"
    
    theory_line = f"Theory ,  1.00 , {Theory_Re_str} , {Theory_Te_str} , {Theory_error:7.2f} ,        - ,         - , {Theory_Be:6.2f} ,         - ,"
    output.append(theory_line)
    
    for res in all_results:
        # Negative sign for nLaps (convention from the base file)
        nLaps_signed = -res['nLaps'] 
        
        line = (
            f"{res['Var']} , {nLaps_signed:5.2f} , {res['Re']:6.2f} , {res['Te']:6.2f} , {res['error']:7.2f} , "
            f"{res['MT/lap']:6.2f} , {res['IDe/lap']:7.2f} , {res['Be']:6.2f} , {res['IPe']:7.2f} ,"
        )
        # Handling NaN values 
        line = line.replace('  nan', '       -').replace(' nan', '      -')
        output.append(line)
        
    return "\n".join(output)

# Execute the main function
results_table = process_circular_task_data(DATA_FILE_NAME, TARGET_BE_VALUES, CYCLE_TIMES_REL)

print(results_table)

Var , nLaps ,      Re ,      Te ,   error ,  MT/lap , IDe/lap ,      Be ,     IPe ,
unit ,   lap ,  pixel ,  pixel ,       % ,  s/lap , bit/lap ,  double ,   bit/s ,
Theory ,  1.00 , 209.50 ,  47.00 ,    3.88 ,        - ,         - ,   1.00 ,         - ,
Rec001 , -10.84 , 213.86 ,  91.79 ,    3.30 ,   1.84 ,    3.87 ,   1.95 ,    2.10 ,
Rec002 , -12.09 , 212.99 ,  98.14 ,    2.13 ,   1.62 ,    3.77 ,   0.73 ,    2.33 ,
Rec003 , -13.35 , 210.66 ,  60.61 ,    1.31 ,   1.47 ,    4.45 ,   0.90 ,    3.02 ,
Rec004 , -12.91 , 210.45 ,  51.96 ,    1.16 ,   1.51 ,    4.67 ,   0.79 ,    3.10 ,
Rec005 , -14.88 , 214.14 , 104.52 ,    2.13 ,   1.34 ,    3.69 ,   0.93 ,    2.75 ,
